In [1]:
import pandas as pd
import regex as re
import os, sys
import PyPDF2

In [2]:
path = "data-csr" ###

In [3]:

# a function to walk through all files in a folder and its subfolders
def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).__next__()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files:                                                                                        
                r.append(os.path.join(subdir, file))                                                                         
    return r

In [4]:
# assign relative directory
directory = os.path.join(sys.path[0], path + "\\raw") ### INPUT FOLDER HERE ###

# list files in directory
files_in_dir = list_files(directory)

# count files in directory
print("Your input directory is:", directory, "\nNumber of files:", len(files_in_dir))

Your input directory is: c:\Users\gparti\GitHub\index-construction\data-csr\raw 
Number of files: 4


In [5]:
# initialize dataframe to hold documents
df = pd.DataFrame(columns=['file'])
df

,file


In [ ]:
# idaig megvan

In [ ]:
# function to read a pdf file and add them to a dataframe
def read_pdf(document, index=0):
  """Read and parse a pdf file.
  This function uses the PyPDF2 package to read and extract 
  the contents of a pdf file, page by page.

  Keyword arguments:
  document -- the document to be read in.
  index -- the index of the document, an integer (default 0)
  """
  # add filename to dataframe
  m = re.search(r"\\(?!.*\\)(.*)_(\d+)(_?.*)?((\.pdf)|(\.txt))", document.lower())
  filename_match = m.group(0)
  filename = re.sub("[\\\/\'\>]", "", filename_match)
  filename = re.sub("\.\w+", "", filename)
  company = m.group(1)
  year = m.group(2)
  type = m.group(3)
  type = re.sub("^_", "", type)
  ext = m.group(4)
  ext = re.sub("\.", "", ext)

  df.loc[index, 'file'] = filename
  df.loc[index, 'company'] = company
  df.loc[index, 'year'] = year
  df.loc[index, 'type'] = type
  if type == "":
    df.loc[index, 'type'] = 'not_csr'
  
  print("Parsing", filename, "...")

  # #####
  # y = re.search("([0-9]{4})", filename)
  # if y is None:
  #   df.loc[index, 'year'] = np.nan
  # else:
  #   df.loc[index, 'year'] = y[0]
  # #####

  # creating a pdf file object
  pdfFileObj = open(document, 'rb') 

  # n = re.search(r"\\(?:.(?!\\))+$", filename_with_path)
  # filename_match = n.group(0)

  # creating a pdf reader object 
  pdfReader = PyPDF2.PdfFileReader(pdfFileObj, strict=False) 
      
  # printing number of pages in pdf file 
  # print("Number of pages:", pdfReader.numPages)
  pages = pdfReader.numPages

  # add page number to dataframe
  # df.loc[index, 'pages'] = pages

  # creating a page object 
  pageObj = pdfReader.getPage(0)

  # extracting text from page 
  # print(pageObj.extractText())

  pages_with_contents = []

  for p in range(pages):
    pageObj = pdfReader.getPage(p)
    page_contents = pageObj.extractText()
    pages_with_contents.append(page_contents)

  # join pages into one document
  contents = " ".join(pages_with_contents)

  #closing the pdf file object 
  pdfFileObj.close() 

  # cleaning
  contents = re.sub("\n", " ", contents)
  contents = re.sub("\.", ". ", contents)
  contents = re.sub("\)", ") ", contents)

  # separates words
  contents = re.sub(r"([a-z])([A-Z])", r"\1 \2", contents)

  # # if need list
  # contents_list = contents.split(' || ')

  # remove symbols
  contents = re.sub(r"[^a-zA-Z0-9]", " ", contents)

  # # lowercase
  contents = contents.lower()

  #remove extra spaces
  contents = re.sub("\s+", " ", contents)

  #from where
  df.loc[index, 'source'] = "manual"

  # add contents to dataframe
  df.loc[index, 'contents'] = contents

  return #print("Done.")